# How To: Run SModelS as a python library

In [1]:
# Set up the path to SModelS installation folder
import sys; sys.path.append("."); import smodels_paths

In [2]:
from imp import reload
from smodels.tools import runtime
from smodels import particlesLoader
from smodels.theory import slhaDecomposer,lheDecomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database
from smodels.tools import coverage
from smodels.tools.smodelsLogging import setLogLevel
setLogLevel("info")

### Main input:

In [3]:
#Define your model (list of rEven and rOdd particles)
runtime.modelFile = 'smodels.share.models.mssm' 
reload(particlesLoader) #Make sure all the model particles are up-to-date
# Path to input file (either a SLHA or LHE file)
slhafile = 'inputFiles/slha/lightEWinos.slha'

### Decompose the input model:

In [4]:
# Set main options for decomposition
sigmacut = 0.01 * fb
mingap = 5. * GeV
# Decompose model (use slhaDecomposer for SLHA input or lheDecomposer for LHE input)
toplist = slhaDecomposer.decompose(slhafile, sigmacut, doCompress=True, doInvisible=True, minmassgap=mingap)

# Access basic information from decomposition, using the topology list and topology objects:
print( "\n Decomposition Results: " )
print( "\t  Total number of topologies: %i " %len(toplist) )
nel = sum([len(top.elementList) for top in toplist])
print( "\t  Total number of elements = %i " %nel )


 Decomposition Results: 
	  Total number of topologies: 51 
	  Total number of elements = 14985 


### Load the Database of experimental results:

In [5]:
# Set the path to the database
database = Database("http://smodels.hephy.at/database/official120")
# Load the experimental results to be used.
# In this case, all results are employed.
listOfExpRes = database.getExpResults()

# Print basic information about the results loaded.
# Count the number of loaded UL and EM experimental results:
nUL, nEM = 0, 0
for exp in listOfExpRes:
    expType = exp.getValuesFor('dataType')[0]
    if expType == 'upperLimit':
        nUL += 1
    elif  expType == 'efficiencyMap':
        nEM += 1
print("\n Loaded Database with %i UL results and %i EM results " %(nUL,nEM))

INFO in databaseObj.loadBinaryFile() in 176: loading binary db file ./official120.pcl format version 201
INFO in databaseObj.loadBinaryFile() in 183: Loaded database from ./official120.pcl in 2.1 secs.



 Loaded Database with 55 UL results and 21 EM results 


### Match the decomposed simplified models with the experimental database of constraints:

In [11]:
# Compute the theory predictions for each experimental result and print them:
print("\n Theory Predictions and Constraints:")
rmax = 0.
bestResult = None
for expResult in listOfExpRes:
    predictions = theoryPredictionsFor(expResult, toplist)
    if not predictions: continue # Skip if there are no constraints from this result
    print('\n %s (%i TeV)' %(expResult.globalInfo.id,expResult.globalInfo.sqrts.asNumber(TeV)))
    for theoryPrediction in predictions:
        dataset = theoryPrediction.dataset
        datasetID = dataset.dataInfo.dataId            
        mass = theoryPrediction.mass
        txnames = [str(txname) for txname in theoryPrediction.txnames]
        PIDs =  theoryPrediction.PIDs         
        print( "------------------------" )
        print( "TxNames = ",txnames )  
        print( "Theory Prediction = ",theoryPrediction.xsection.value )  #Signal cross section
        # Get the corresponding upper limit:
        print( "UL for theory prediction = ",theoryPrediction.upperLimit )
        # Compute the r-value
        r = theoryPrediction.getRValue()
        print( "r = ",r )
        #Compute likelihhod and chi^2 for EM-type results:
        if dataset.dataInfo.dataType == 'efficiencyMap':
            theoryPrediction.computeStatistics()
            print( 'Chi2, likelihood=', theoryPrediction.chi2, theoryPrediction.likelihood )
        if r > rmax:
            rmax = r
            bestResult = expResult.globalInfo.id

# Print the most constraining experimental result
print( "\nThe largest r-value (theory/upper limit ratio) is ",rmax )
if rmax > 1.:
    print( "(The input model is likely excluded by %s)" %bestResult )
else:
    print( "(The input model is not excluded by the simplified model results)" )


 Theory Predictions and Constraints:

 ATLAS-SUSY-2015-06 (13 TeV)
------------------------
TxNames =  ['T1']
Theory Prediction =  5.19E-06 [pb]
UL for theory prediction =  1.79E+00 [fb]
r =  0.002901393530776837
Chi2, likelihood= 2.3776849368423356 0.007169156710956845

 CMS-PAS-SUS-17-004 (13 TeV)
------------------------
TxNames =  ['TChiWH']
Theory Prediction =  6.10E-04 [pb]
UL for theory prediction =  3.79E+02 [fb]
r =  0.001609898825011838
------------------------
TxNames =  ['TChiWZ']
Theory Prediction =  2.79E-04 [pb]
UL for theory prediction =  1.18E+02 [fb]
r =  0.002362228895825259

 CMS-SUS-16-033 (13 TeV)
------------------------
TxNames =  ['T2']
Theory Prediction =  2.54E-05 [pb]
UL for theory prediction =  3.60E+01 [fb]
r =  0.0007039737677262512

 CMS-SUS-16-034 (13 TeV)
------------------------
TxNames =  ['TChiWZ']
Theory Prediction =  2.79E-04 [pb]
UL for theory prediction =  2.32E+02 [fb]
r =  0.0012033340641033744

 CMS-SUS-16-036 (13 TeV)
----------------------


 CMS-SUS-13-012 (8 TeV)
------------------------
TxNames =  ['T1', 'T1bbbb', 'T1tttt', 'T2', 'T5WWoff', 'TChiWW', 'TChiWZ']
Theory Prediction =  1.13E-04 [pb]
UL for theory prediction =  1.22E+00 [fb]
r =  0.09235472614936503
Chi2, likelihood= 0.18657363971347962 0.0034837107545883945

 CMS-SUS-13-013 (8 TeV)
------------------------
TxNames =  ['T1tttt']
Theory Prediction =  6.70E-04 [pb]
UL for theory prediction =  3.35E+01 [fb]
r =  0.020002345839295096

 CMS-SUS-13-013 (8 TeV)
------------------------
TxNames =  ['T1tttt']
Theory Prediction =  4.71E-06 [pb]
UL for theory prediction =  2.58E-01 [fb]
r =  0.0182883542214002
Chi2, likelihood= 0.018456286893814067 0.10912239638341135

 CMS-SUS-13-019 (8 TeV)
------------------------
TxNames =  ['T1']
Theory Prediction =  2.03E-04 [pb]
UL for theory prediction =  7.45E+01 [fb]
r =  0.002718495202475519
------------------------
TxNames =  ['T1tttt']
Theory Prediction =  6.70E-04 [pb]
UL for theory prediction =  7.57E+01 [fb]
r =  0.0088

### Check for simplified models in the input model which were not tested by the Database:

In [7]:
#Find out missing topologies for sqrts=8*TeV:
uncovered = coverage.Uncovered(toplist,sqrts=8.*TeV)
#Print uncovered cross-sections:
print( "\nTotal missing topology cross section (fb): %10.3E\n" %(uncovered.getMissingXsec()) )
print( "Total cross section where we are outside the mass grid (fb): %10.3E\n" %(uncovered.getOutOfGridXsec()) )
print( "Total cross section in long cascade decays (fb): %10.3E\n" %(uncovered.getLongCascadeXsec()) )
print( "Total cross section in decays with asymmetric branches (fb): %10.3E\n" %(uncovered.getAsymmetricXsec()) )

#Print some of the missing topologies:
print( 'Missing topologies (up to 3):' )
for topo in uncovered.missingTopos.topos[:3]:
    print( 'Topology:',topo.topo )
    print( 'Contributing elements (up to 2):' )
    for el in topo.contributingElements[:2]:
        print( el,'cross-section (fb):', el.missingX )

#Print elements with long cascade decay:
print( '\nElements outside the grid (up to 2):' )
for topo in uncovered.outsideGrid.topos[:2]:
    print( 'Topology:',topo.topo )
    print( 'Contributing elements (up to 4):' )
    for el in topo.contributingElements[:4]:
        print( el,'cross-section (fb):', el.missingX )
        print( '\tmass:',el.getMasses() )



Total missing topology cross section (fb):  3.717E+03

Total cross section where we are outside the mass grid (fb):  9.989E+01

Total cross section in long cascade decays (fb):  8.992E+02

Total cross section in decays with asymmetric branches (fb):  2.790E+03

Missing topologies (up to 3):
Topology: [[],[]](MET,MET)
Contributing elements (up to 2):
[[],[]] cross-section (fb): 0.45069449699999997
[[],[]] cross-section (fb): 0.11641645217403078
Topology: [[],[[W]]](MET,MET)
Contributing elements (up to 2):
[[],[[W+]]] cross-section (fb): 0.11376799860548632
[[],[[W+]]] cross-section (fb): 0.05277778742404712
Topology: [[],[[Z]]](MET,MET)
Contributing elements (up to 2):
[[],[[Z]]] cross-section (fb): 0.8060860593513246
[[],[[Z]]] cross-section (fb): 0.07370694075178172

Elements outside the grid (up to 2):
Topology: [[[W]],[[Z]]](MET,MET)
Contributing elements (up to 4):
[[[W+]],[[Z]]] cross-section (fb): 0.3637955190752928
	mass: [[2.93E+02 [GeV], 6.81E+01 [GeV]], [2.66E+02 [GeV], 6.8